In [6]:
import os
import pandas as pd
import sqlalchemy as sa

#-----------------------------------------
# CONFIG
#-----------------------------------------
root_path = r"C:\Users\Public\Documents\East Africa Airbnb Advance SQL-Power Bi Project\Kenya"

  # folder that contains the 5 subfolders
folders = ["Kiambu", "Nakuru", "Nairobi", "Mombasa", "Kisumu"]

# SQL Server connection string
connection_string = (
    "mssql+pyodbc://@LPJZ5KFY3/Airbnb_data"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)


engine = sa.create_engine(connection_string)

#-----------------------------------------
# STEP 1: Build a dictionary of suffixes
#-----------------------------------------
suffix_groups = {}

for folder in folders:
    folder_path = os.path.join(root_path, folder)

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):

            # Extract suffix: everything after the first underscore
            suffix = file.split("_", 1)[1]   # keeps "occupancy_last_12_month.csv"

            # Add to dictionary
            if suffix not in suffix_groups:
                suffix_groups[suffix] = []

            suffix_groups[suffix].append(os.path.join(folder_path, file))

print("Detected unified suffixes:", list(suffix_groups.keys()))

#-----------------------------------------
# STEP 2: Union files by suffix
#-----------------------------------------
for suffix, filepaths in suffix_groups.items():

    print(f"\nUnioning group: {suffix}")
    df_list = []

    for file in filepaths:
        print(f"   Reading: {file}")
        try:
            df = pd.read_csv(file)

            # CLEAN HEADER NAMES
            df.columns = df.columns.str.strip()             # remove trailing spaces
            df.columns = df.columns.str.replace(" ", "_")   # normalize
            df.columns = df.columns.str.replace("-", "_")
            df.columns = df.columns.str.replace("/", "_")

            df_list.append(df)

        except Exception as e:
            print(f"   ERROR reading {file}: {e}")
            continue

    if not df_list:
        print("   No valid CSV files. Skipping.")
        continue

    # Union
    union_df = pd.concat(df_list, ignore_index=True)

    # Clean final columns again to ensure uniqueness
    union_df.columns = union_df.columns.str.strip().str.replace(" ", "_")

    table_name = os.path.splitext(suffix)[0].replace(".", "_")

    print(f"   Loading into SQL table: {table_name}")

    union_df.to_sql(
        table_name,
        engine,
        if_exists="replace",
        index=False,
        chunksize=5000
    )
print("\nAll files unioned and loaded to SQL Server successfully!")

Detected unified suffixes: ['occupancyBedrooms_last_12_month.csv', 'occupancyDemand_last_12_month.csv', 'occupancyFuture_next_30_days.csv', 'occupancyLength_last_12_month.csv', 'occupancy_last_12_month.csv', 'rateByBedroom_last_12_month.csv', 'rateByDailyAverage_last_12_month.csv', 'rateByDay_last_12_month.csv', 'rateByPriceTier_last_12_month.csv', 'rateFuture_next_180_days.csv', 'revenueAverage_last_12_month.csv', 'revenueByBedroom_last_12_month.csv', 'revenueByPercentile_last_12_month.csv', 'revenueByRealEstateType_last_12_month.csv', 'revparHighestInYear_last_12_month.csv', 'revparInAdvance_last_12_month.csv', 'revpar_last_12_month.csv', 'occupancyBedrooms_last_12_month (1).csv', 'occupancyFuture_next_30_days (1).csv']

Unioning group: occupancyBedrooms_last_12_month.csv
   Reading: C:\Users\Public\Documents\East Africa Airbnb Advance SQL-Power Bi Project\Kenya\Kiambu\Kiambu_occupancyBedrooms_last_12_month.csv
   Reading: C:\Users\Public\Documents\East Africa Airbnb Advance SQL-Powe